In [ ]:
# 📊 Projet : Intelligence Artificielle & Pricing d'Obligations High Yield
**Auteur :** Maéva Cavaleri (Télécom SudParis)  
**Sujet :** Imputation de données manquantes par Système de Recommandation (Matrix Factorization) et Analyse de Valeur Relative (Rich/Cheap).

In [ ]:
## 1. Introduction : Le Problème de l'Illiquidité en Finance

### 1.1 Le Marché High Yield (HY)
Les obligations "High Yield" (obligations à haut rendement) sont émises par des entreprises dont la note de crédit est jugée risquée (inférieure à **BBB-**). En échange de ce risque de défaut plus élevé, l'investisseur reçoit un **Spread** (une prime de rendement) important.

### 1.2 Le défi des données manquantes (NaN)
Contrairement aux actions cotées en continu, les obligations HY s'échangent de gré à gré (**OTC - Over The Counter**). Conséquence :
*   De nombreux titres ne s'échangent pas tous les jours.
*   Il y a des **"NaN" (Not a Number)** dans nos bases de données : ce sont des trous d'information.
*   **Le problème du Quant :** Comment estimer la valeur d'une obligation le mardi si sa dernière transaction date du vendredi ?

In [ ]:
## 2. Génération des Données : Création d'un Monde Synthétique

Pour tester notre IA, nous créons un univers financier contrôlé. Nous utilisons un **Modèle Factoriel**.

### 2.1 La Formule Stochastique
Le spread d'une obligation $i$ au temps $t$ est généré par :
$$Spread_{i,t} = Base + (\beta_i \times Marché_t) + Secteur_{s,t} + \epsilon_{i,t}$$

*   **Base (350 bps) :** Le niveau de risque "plancher" de notre univers.
*   **Marché ($M_t$) :** Une marche aléatoire (`np.cumsum`) qui représente l'économie globale.
*   **Secteur ($S_{s,t}$) :** Une tendance propre à chaque industrie (Tech, Energie...). C'est ce qui crée la **corrélation**.
*   **Beta ($\beta_i$) :** La sensibilité de l'obligation (agressive ou défensive).
*   **$\epsilon$ (Bruit Idiosyncratique) :** L'aléa pur de l'entreprise.

In [2]:
# CODE : Génération du "monde complet"
import sys, os
sys.path.append('../') # On pointe vers le dossier src/
from src.data_generation import BondDataGenerator

gen = BondDataGenerator('../config/settings.yaml')
df_full, metadata = gen.generate()

# On crée la "Matrice de Gruyère" (on cache 75% des données)
df_incomplete = gen.apply_illiquidity(df_full)

ModuleNotFoundError: No module named 'src'

In [ ]:
## 3. Le Cerveau : Matrix Factorization (SVD Itérative)

C'est ici que nous utilisons les techniques des systèmes de recommandation (type Netflix/Amazon).

### 3.1 Pourquoi la SVD ?
La **Singular Value Decomposition** décompose la matrice des spreads en **Facteurs Latents**.
*   Elle découvre que les obligations ne bougent pas au hasard mais suivent des "thèmes" cachés.
*   Si le thème "Pétrole" baisse, la SVD va automatiquement baisser le prix de toutes les obligations liées au pétrole, même celles qui n'ont pas de prix affiché aujourd'hui.

### 3.2 L'importance du Centrage
Avant de lancer l'IA, on soustrait la moyenne de chaque obligation. 
**Pourquoi ?** Pour que le modèle ne compare pas des niveaux de prix (200 vs 600) mais des **variations de prix**. On veut que l'IA apprenne les corrélations, pas les moyennes.

### 3.3 L'algorithme Itératif
1.  On remplit les trous par une valeur simple (moyenne).
2.  On décompose avec la SVD pour extraire la structure du marché.
3.  On utilise cette structure pour prédire plus précisément les trous.
4.  On recommence $X$ fois pour affiner la précision.

In [ ]:
from src.models import OASImputer
imputer = OASImputer(n_factors=15) # On cherche les 15 thèmes majeurs du marché

# Imputation par IA
df_ml = imputer.matrix_factorization_fill(df_incomplete, iterations=5)

In [ ]:
## 4. Évaluation : L'IA est-elle fiable ?

Nous utilisons le **RMSE (Root Mean Squared Error)**. On calcule l'écart entre la **Vraie Valeur** (que nous avons générée) et la **Valeur Prédite** par l'IA, uniquement sur les points qui étaient manquants.

$$RMSE = \sqrt{\frac{1}{N} \sum (Vrai - Prédit)^2}$$

### Analyse par Secteur
Toutes les industries ne sont pas égales. Certains secteurs sont très corrélés (faciles à prédire), d'autres sont très volatils. L'analyse sectorielle permet au gérant de savoir quel crédit accorder au modèle.

In [ ]:
from src.evaluation import ModelEvaluator
eval = ModelEvaluator(df_full, df_incomplete, metadata)
print(f"Précision globale : {eval.calculate_rmse(df_ml):.2f} bps")

In [ ]:
## 5. Application Métier : Stratégie Rich / Cheap

C'est l'étape finale : **faire de l'argent**. 

### 5.1 La notion de Fair Value
Le prix prédit par l'IA est la **"Fair Value"** (la juste valeur théorique). Si le marché s'éloigne trop de cette valeur, il y a une anomalie.

### 5.2 Les Signaux de Trading
On calcule : $Ecart = Spread_{Marché} - Spread_{IA}$.

1.  **CHEAP (Achat) :** Si $Ecart > 15$ bps. Le marché demande un rendement de 500 alors que l'IA dit que le risque n'en vaut que 450. L'obligation est "bradée", on l'achète.
2.  **RICH (Vente) :** Si $Ecart < -15$ bps. Le marché est trop optimiste, le rendement est trop faible pour le risque réel. On vend.

**Pourquoi 15 bps ?** Pour couvrir les frais de transaction (Bid-Ask Spread) et l'incertitude du modèle.

In [ ]:
from src.trading_strategy import TradingStrategy
strat = TradingStrategy(threshold=15)
signals, _ = strat.generate_signals(df_incomplete, df_ml)

In [ ]:
## 6. Conclusion & Perspectives (Lien avec Cassiopée)

Ce projet démontre qu'une approche par **Réduction de Dimension (SVD)** permet de vaincre l'illiquidité. 